# Roberta finetuning

In [ ]:
!pip install transformers datasets torch scikit-learn
!pip install evaluate

In [ ]:
import pandas as pd
df=pd.read_csv('/kaggle/input/ai-text/ai_press_releases.csv')
df=df.dropna()
human=df['non_chat_gpt_press_release']
ai=df['chat_gpt_generated_release']
hu=[]
a=[]
for i in human:
    l=list(i.split('. '))
    hu.extend(l)
for i in ai:
    l=list(i.split('. '))
    a.extend(l)

In [ ]:
print(len(a))
print(len(hu))

In [ ]:
ap=a.copy()
a.extend(hu)
texts=a
labels=[0 if i<len(ap) else 1 for i in range(len(texts))]
from sklearn.model_selection import train_test_split

# 1) 먼저 train_temp(80%)와 test(20%) 분할
texts_train_val, texts_test, labels_train_val, labels_test = train_test_split(
    texts,
    labels,
    test_size=0.2,       # 전체의 20%
    random_state=42,
    stratify=labels      # 레이블 비율 유지
)

# 2) train_temp을 다시 train(75% of temp → 60% 전체)와 val(25% of temp → 20% 전체)로 분할
texts_train, texts_val, labels_train, labels_val = train_test_split(
    texts_train_val,
    labels_train_val,
    test_size=0.25,      # train_temp의 25% → 전체의 0.2
    random_state=42,
    stratify=labels_train_val
)

print(f"Train: {len(texts_train)} samples")
print(f"Valid: {len(texts_val)} samples")
print(f"Test : {len(texts_test)} samples")

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from datasets import Dataset
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score

# — 데이터 & DataLoader 준비 (이미 정의된 texts_train/val/test, labels_* 목록 사용) —
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def collate_fn(batch):
    enc = tokenizer(
        [x["text"] for x in batch],
        padding="longest",
        truncation=True,
        max_length=256,
        return_tensors="pt"
    )
    enc["labels"] = torch.tensor([x["label"] for x in batch], dtype=torch.long)
    return enc

train_ds = Dataset.from_dict({"text": texts_train, "label": labels_train})
val_ds   = Dataset.from_dict({"text": texts_val,   "label": labels_val})
test_ds  = Dataset.from_dict({"text": texts_test,  "label": labels_test})

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False, collate_fn=collate_fn)

# — 모델/옵티마이저 세팅 —
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2).to(device)
optim  = AdamW(model.parameters(), lr=2e-5)

num_epochs = 8

for epoch in range(1, num_epochs+1):
    # ── 1) TRAIN ───────────────────────────────────────────
    model.train()
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs} [TRAIN]")
    for batch in train_loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss    = outputs.loss
        optim.zero_grad()
        loss.backward()
        optim.step()
        gpu_mem = torch.cuda.memory_allocated(device) // (1024**2)
        train_loop.set_postfix(loss=f"{loss.item():.4f}", gpu_mem=f"{gpu_mem}MiB")

    # ── 2) VALIDATION ──────────────────────────────────────
    model.eval()
    all_preds, all_labels = [], []
    val_loop = tqdm(val_loader, desc=f"Epoch {epoch}/{num_epochs} [VAL]  ")
    with torch.no_grad():
        for batch in val_loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**batch).logits
            preds  = torch.argmax(logits, dim=-1).cpu().tolist()
            labels = batch["labels"].cpu().tolist()
            all_preds += preds
            all_labels += labels
    val_acc = accuracy_score(all_labels, all_preds)
    val_f1  = f1_score(all_labels, all_preds, average="weighted")
    print(f"→ Validation | Acc: {val_acc:.4f}, F1: {val_f1:.4f}")

    # ── 3) TEST ────────────────────────────────────────────
    all_preds, all_labels = [], []
    test_loop = tqdm(test_loader, desc=f"Epoch {epoch}/{num_epochs} [TEST] ")
    with torch.no_grad():
        for batch in test_loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**batch).logits
            preds  = torch.argmax(logits, dim=-1).cpu().tolist()
            labels = batch["labels"].cpu().tolist()
            all_preds += preds
            all_labels += labels
    test_acc = accuracy_score(all_labels, all_preds)
    test_f1  = f1_score(all_labels, all_preds, average="weighted")
    print(f"→ Test       | Acc: {test_acc:.4f}, F1: {test_f1:.4f}")

    # ── 4) MODEL SAVE ──────────────────────────────────────
    save_dir = f"/kaggle/working/checkpoint-epoch{epoch}"
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
    print(f"→ Model & Tokenizer saved to: {save_dir}\n")
